# Closed-loop optimization

### The surrogate model can be changed

In [ ]:
# import all required packages 
import math
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# for RF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

#for GP
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,Matern, ConstantKernel as C

# for the NN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

import xlrd

import matplotlib.pyplot as plt

### Feature selection

In [ ]:
# Get the inputs and the outputs ready - the features can be generated using the separate file Preprocessing.ipynb

full_dataset = data_2_sterimol
full_dataset = np.append(full_dataset,dft_data,axis=1)
full_dataset = np.append(full_dataset,fp_pca_data,axis=1)
full_dataset = np.append(full_dataset,dataset_one_hot,axis=1)
full_dataset = np.append(full_dataset,NBO_data,axis=1)
full_dataset = np.append(full_dataset,chelpg_data,axis=1)
full_dataset = np.append(full_dataset,vol_bur,axis=1)
full_dataset = np.append(full_dataset,n_proton,axis=1)
full_dataset = np.append(full_dataset,yield_numerical,axis=1)

feature_column_names = ['N_sterimol_l','N_sterimol_b1','N_sterimol_b5','R_sterimol_l','R_sterimol_b1','R_sterimol_b5',
                       'homo', 'lumo',
                       'ligands_PCA_1',' ligands_PCA_2',' ligands_PCA_3','ligands_PCA_4','ligands_PCA_5','ligands_PCA_6','ligands_PCA_7','ligands_PCA_8','ligands_PCA_9','ligands_PCA_10','ligands_PCA_11','ligands_PCA_12','ligands_PCA_13','ligands_PCA_14','ligands_PCA_15','ligands_PCA_16','ligands_PCA_17','ligands_PCA_18','ligands_PCA_19','ligands_PCA_20','ligands_PCA_21','ligands_PCA_22','ligands_PCA_23','ligands_PCA_24','ligands_PCA_25','ligands_PCA_26','ligands_PCA_27','ligands_PCA_28','ligands_PCA_29','ligands_PCA_30',
                       'precat_PCA_1',' precat_PCA_2',' precat_PCA_3','precat_PCA_4','precat_PCA_5','precat_PCA_6','precat_PCA_7','precat_PCA_8','precat_PCA_9','precat_PCA_10','precat_PCA_11','precat_PCA_12','precat_PCA_13','precat_PCA_14','precat_PCA_15','precat_PCA_16','precat_PCA_17','precat_PCA_18','precat_PCA_19','precat_PCA_20','precat_PCA_21','precat_PCA_22','precat_PCA_23','precat_PCA_24','precat_PCA_25','precat_PCA_26','precat_PCA_27','precat_PCA_28','precat_PCA_29','precat_PCA_30',
                       'boronic_PCA_1',' boronic_PCA_2',' boronic_PCA_3','boronic_PCA_4','boronic_PCA_5','boronic_PCA_6','boronic_PCA_7','boronic_PCA_8','boronic_PCA_9','boronic_PCA_10','boronic_PCA_11','boronic_PCA_12','boronic_PCA_13','boronic_PCA_14','boronic_PCA_15','boronic_PCA_16','boronic_PCA_17','boronic_PCA_18','boronic_PCA_19','boronic_PCA_20','boronic_PCA_21','boronic_PCA_22','boronic_PCA_23','boronic_PCA_24','boronic_PCA_25','boronic_PCA_26','boronic_PCA_27','boronic_PCA_28','boronic_PCA_29','boronic_PCA_30',
                       'OHE_lig_1',' OHE_lig_2',' OHE_lig_3','OHE_lig_4','OHE_lig_5','OHE_lig_6','OHE_lig_7','OHE_lig_9','OHE_lig_10','OHE_lig_11','OHE_lig_12','OHE_lig_13','OHE_lig_14','OHE_lig_15','OHE_lig_16','OHE_lig_17','OHE_lig_18','OHE_lig_19','OHE_lig_20','OHE_lig_21','OHE_lig_22','OHE_lig_23','OHE_lig_24','OHE_lig_25','OHE_lig_26','OHE_lig_27','OHE_lig_28','OHE_lig_29','OHE_lig_30',' OHE_lig_31',' OHE_lig_32',
                       'OHE_precat_1','OHE_precat_2','OHE_precat_3',
                       'OHE_bor_1','OHE_bor_2',
                       'NBO_N','NBO_OR','NBO_O','NBO_AcO',
                       'chelpg_N','chelpg_OR','chelpg_O','chelpg_AcO',
                       'vol_bur',
                       'n_proton',
                       'Yield']

full_dataset_input = pd.DataFrame(full_dataset)
full_dataset_input.columns = feature_column_names

print('Shape of the featurized dataset:',full_dataset_input.shape)

### To allow for fair comparison across different models when conducting 10 repetitions, the data used for initialization was set a priori for various initialization strategies


In [ ]:
# using 5 datapoints for initialization
#random_initialization = [183, 127, 107, 118, 48, 16, 71, 89, 90, 64, 42, 70, 29, 4, 42, 139, 89, 51, 167]
#random_initialization = [44, 92, 179, 151, 129, 170, 110, 177, 133, 165, 151, 162, 102, 3, 78, 99, 70, 37, 55]
#random_initialization = [161, 23, 86, 39, 136, 114, 87, 177, 3, 2, 158, 70, 156, 130, 85, 106, 155, 114, 142]
#random_initialization = [140, 47, 117, 63, 33, 73, 41, 87, 173, 67, 80, 93, 182, 112, 1, 127, 134, 20, 31]
#random_initialization = [84, 5, 4, 90, 86, 165, 68, 85, 29, 184, 113, 43, 178, 63, 25, 12, 37, 180, 93]
#random_initialization = [71, 70, 179, 2, 182, 19, 182, 70, 184, 170, 18, 132, 74, 107, 171, 90, 116, 123, 162]
#random_initialization = [76, 184, 123, 40, 24, 83, 48, 98, 63, 14, 124, 123, 51, 128, 16, 137, 1, 74, 44]
#random_initialization = [77, 41, 131, 114, 50, 65, 14, 56, 95, 71, 167, 137, 57, 135, 4, 165, 2, 105, 89]
#random_initialization = [165, 81, 147, 19, 117, 62, 80, 124, 139, 7, 28, 107, 149, 55, 78, 44, 37, 135, 32]
#random_initialization = [123, 52, 43, 101, 43, 98, 137, 93, 87, 85, 10, 152, 131, 53, 49, 76, 100, 44, 57]

#using 10 datapoints for initialization
#random_initialization = [162, 129,  89, 40,124,  16, 165, 100, 177, 108, 138, 104, 103,  79, 152, 137, 112,44, 124, 175, 164, 149,   9,  36]
#random_initialization = [161,  98, 147, 158, 118, 180, 184,  93,  57,  31, 136, 103, 151, 112, 162,  57,   1,  74,  62, 112,   7,  55,  99,   9]
#random_initialization = [155,  23, 169,   2,  80,  56, 179, 139,  73,  42,  41,  83, 168, 64, 155,  23,  54, 171,  42, 166, 122, 106,   8, 147]
#random_initialization = [ 68,  54, 163, 175,  62, 108,  85,  24, 180, 116, 123,  25, 149, 87,  79,   1,  47, 104, 113,  69,   0, 175,  85,  79]
#random_initialization = [ 52, 127,  89,  14,  64,   3,  41,  55,  84, 165,   0,  28,  30, 20, 124,  72,  52,  68,   9,  49, 184, 139, 127, 138]
#random_initialization = [ 95,   9,  86,  82, 177,  85, 183, 133,  63, 155,  76,   6, 173, 71,  15, 179,  51,  28,  10, 102,  57,  73,   1, 129]
#random_initialization = [ 95, 183, 118, 110,  61,  24,  82, 143, 150,  37, 138, 166,  18, 131,  35,  34, 159,  23, 139,  68,  22,  18, 153, 120]
#random_initialization = [ 33,  82, 161,  49,  12, 116,  40,  87,  50, 164, 177,  37, 126, 166,  32, 127, 180,  68,  74, 134, 103,  35,   9,  24]
#random_initialization = [131,  78,  13,  81, 139,  30,  79,  25,  29,  73,  46,  50,  19, 60, 151, 108, 121, 134,  72,  79,  43, 161, 132,  97]
#random_initialization = [ 22,   2, 129,  68, 161,   7,  63,  24,  85,  18,  92, 162, 175, 45, 153,   0,  67,  73,  92,  69, 153,  61,  10, 149]

#print(len(random_initialization))
# using 15 datapoints for initialization
#random_initialization = [116, 78, 65, 148, 108, 175, 149, 27, 81, 38, 119, 26, 47, 136, 96, 87, 47, 2, 62, 17, 159, 91, 37, 114, 70, 24, 168, 11, 50]
#random_initialization = [59, 167, 175, 24, 116, 111, 100, 113, 32, 167, 52, 4, 140, 131, 87, 109, 103, 145, 70, 0, 76, 149, 29, 46, 39, 111, 19, 41, 41]
#random_initialization = [85, 14, 3, 111, 66, 59, 25, 170, 39, 93, 61, 138, 93, 136, 74, 1, 119, 68, 79, 108, 137, 25, 77, 156, 164, 118, 131, 164, 49]
#random_initialization = [118, 99, 134, 56, 92, 80, 157, 183, 70, 133, 120, 105, 102, 100, 56, 79, 112, 106, 37, 68, 118, 27, 0, 162, 97, 54, 130, 163, 61]
#random_initialization = [10, 6, 6, 31, 35, 133, 142, 104, 27, 0, 72, 133, 147, 53, 137, 148, 23, 61, 137, 51, 47, 165, 8, 77, 16, 81, 31, 22, 171]
#random_initialization = [92, 77, 183, 142, 37, 109, 180, 25, 82, 65, 134, 175, 79, 175, 113, 120, 179, 97, 158, 29, 181, 104, 67, 137, 30, 132, 73, 137, 30]
#random_initialization = [4, 11, 119, 77, 10, 73, 99, 9, 54, 56, 159, 140, 51, 124, 20, 66, 38, 159, 143, 106, 170, 87, 86, 50, 142, 43, 58, 136, 91]
#random_initialization = [129, 59, 49, 11, 98, 138, 174, 12, 87, 131, 146, 127, 8, 26, 102, 41, 8, 118, 170, 148, 177, 69, 13, 143, 54, 123, 172, 182, 73]
#random_initialization = [83, 182, 181, 45, 150, 20, 32, 75, 149, 92, 67, 71, 155, 85, 61, 15, 60, 93, 37, 28, 129, 47, 125, 49, 80, 26, 111, 44, 115]
#random_initialization = [93, 3, 82, 14, 89, 29, 21, 104, 155, 105, 0, 87, 106, 147, 20, 136, 82, 136, 159, 8, 96, 124, 24, 50, 25, 121, 43, 8, 44]

#using 20 datapoints for initialization
#random_initialization = [86, 13, 25, 149, 125, 141, 96, 39, 138, 59, 64, 104, 82, 133, 88, 27, 52, 60, 159, 105, 122, 176, 24, 152, 134, 68, 52, 102, 173, 28, 102, 63, 50, 18]
#random_initialization = [150, 103, 52, 7, 3, 72, 48, 115, 7, 67, 97, 184, 164, 26, 55, 173, 3, 70, 74, 114, 26, 89, 84, 158, 155, 97, 99, 174, 136, 111, 16, 72, 134, 20]
#random_initialization = [179, 14, 101, 94, 14, 161, 147, 118, 66, 155, 80, 40, 154, 154, 28, 46, 155, 31, 125, 5, 15, 13, 53, 131, 131, 12, 71, 112, 114, 114, 140, 141, 174, 91]
#random_initialization = [91, 92, 136, 131, 177, 53, 58, 48, 14, 41, 51, 102, 125, 149, 28, 179, 117, 175, 174, 183, 111, 35, 137, 143, 24, 118, 103, 68, 70, 0, 175, 183, 155, 109]
#random_initialization = [152, 177, 94, 112, 98, 151, 106, 106, 11, 81, 36, 58, 54, 115, 105, 38, 74, 31, 135, 38, 177, 63, 120, 167, 156, 142, 63, 176, 180, 40, 61, 15, 114, 30]
#random_initialization = [77, 112, 90, 28, 76, 50, 48, 90, 131, 103, 34, 28, 32, 69, 22, 118, 129, 19, 115, 22, 117, 116, 148, 108, 99, 90, 128, 174, 5, 105, 70, 172, 40, 94]
#random_initialization = [139, 131, 154, 5, 13, 12, 124, 150, 155, 7, 24, 173, 183, 142, 86, 28, 148, 53, 6, 107, 182, 1, 9, 143, 55, 9, 166, 85, 156, 61, 16, 16, 26, 3]
#random_initialization = [182, 62, 184, 88, 153, 20, 100, 169, 171, 66, 183, 89, 107, 18, 154, 68, 26, 80, 95, 106, 119, 23, 3, 167, 110, 79, 6, 93, 141, 7, 113, 83, 74, 136]
#random_initialization = [49, 154, 13, 64, 159, 87, 143, 127, 67, 69, 131, 181, 140, 81, 101, 53, 109, 161, 121, 155, 20, 59, 55, 121, 101, 32, 173, 120, 174, 32, 108, 106, 45, 160]
#random_initialization = [85, 174, 147, 104, 37, 173, 27, 181, 152, 127, 47, 6, 166, 156, 182, 54, 149, 97, 81, 72, 57, 9, 170, 174, 110, 171, 148, 55, 142, 25, 118, 130, 170, 105]

#local initialization
#random_initialization =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24] 
#random_initialization =  [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48] 
#random_initialization = [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72] 
#random_initialization = [73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96] 
#random_initialization = [97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120] 
#random_initialization = [121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144] 
#random_initialization = [145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168] 
#random_initialization = [161, 162, 163, 164, 165, 166, 167, 168,169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184]

initial_data = full_dataset_input.loc[random_initialization]

lookup_table = full_dataset_input.drop(random_initialization)

lookup_table = shuffle(lookup_table)

### Illustration of the data for initialization in the  chemical space

In [ ]:
pca_illustrate = PCA(n_components=2)
fingerprints_pca_2 = pca_illustrate.fit_transform(full_dataset_input)
print(fingerprints_pca_2.shape)
fingerprints_pca_2 = np.array(fingerprints_pca_2)

#let's pull out the PC from the initial dataset
fingerprints_init_pca = fingerprints_pca_2[random_initialization]
print(fingerprints_init_pca.shape)

# let's delete the initial data from the whole lookup table
fingerprints_lookupTable_pca = np.delete(fingerprints_pca_2, random_initialization, 0)
print(fingerprints_lookupTable_pca.shape)

plt.title(label="Local Initialization",fontsize=16)

plt.scatter(fingerprints_lookupTable_pca[:,0], fingerprints_lookupTable_pca[:,1],label='Lookup data',color='dodgerblue')
plt.scatter(fingerprints_init_pca[10:,0],fingerprints_init_pca[10:,1] ,color='dodgerblue')
plt.scatter(fingerprints_init_pca[:10,0],fingerprints_init_pca[:10,1] ,label='Initialization data',color='red')

plt.xlabel('PC1',fontsize=14)
plt.ylabel('PC2',fontsize=14)

plt.rc('xtick', labelsize=13)    
plt.rc('ytick', labelsize=13)
plt.rc('legend', fontsize=13) 

plt.legend()

In [ ]:
# Let's split the large dataset into the data which we will use for building the initial model

lookup_output = lookup_table['Yield']
print(lookup_output.shape)

lookup_input = lookup_table.drop(columns='Yield')
print(lookup_input.shape)

### Starting the closed-loop optimization

In [ ]:
dataset_shape = []

RMSE = []

lookup_input = np.array(lookup_input)

lookup_output = np.array(lookup_output)

lookup_output = lookup_output.reshape(-1,1)

pred_yield_development = []

# set the very initial train features and labels
train_features = train_features_init
train_labels = train_labels_init

current_maximum =[]

for i in range (153):
    train_x = train_features 
    train_y = train_labels
    
    # train the model
    regressor = RandomForestRegressor(n_estimators=200, random_state=0)
    regressor.fit(train_x, train_y)
    test_predictions = regressor.predict(test_features_init)
    
    # evaluate the model
    rmse = metrics.mean_absolute_error(test_labels_init, test_predictions)
    RMSE.append(rmse)
    
    # predictions over the unexplored area and evaluate the best predictions
    lookup_predictions = regressor.predict(lookup_input)
    
    # append the current best datapoint to the table current_maximum
    current_maximum.append(np.max(train_labels))

    # evaluate the highest predicted datapoint given the whole chemical space and the trained model
    best = np.max(lookup_predictions)
    List = lookup_predictions.tolist()
    index = List.index(best)
    
    new_input = lookup_input[index].reshape(1,-1)
    new_output = lookup_output[index].reshape(1,-1)
    
    train_features = np.append(train_features,new_input,axis=0)
    train_labels = np.append(train_labels,lookup_output[index],axis=0)
    
    lookup_input = np.delete(lookup_input, [index], 0)
    lookup_output = np.delete(lookup_output, [index], 0)
    
    # append the best predicted yield to the table pre_yield_development
    pred_yield_development.append(new_output)
    
    print(i,':The current best datapoint is:',np.max(train_labels))